# IYP Query API - Essential Features Test

Quick demonstration of the core capabilities

In [1]:
# Setup - Import and Connect
from iyp_query import connect, Q, And, Or
import pandas as pd

# Connect to database
iyp = connect('bolt+s://iyp.christyquinn.com:7687', 'neo4j', 'lewagon25omgbbq')
print("✅ Connected to IYP database")

✅ Connected to IYP database


query = (iyp.builder()          # Start query
         .find('AS', asn=15169)  # Find node
         .with_organizations()    # Add related data
         .execute())  

In [38]:
cypher, param = query.to_cypher()

In [ ]:
# Query 1: Find AS 216139 and its managing organizations
# Plain English: "Find the Autonomous System number 216139 and show me
# which organization manages/owns it, including the organization's details"
query1 = (iyp.builder()
          .find('AS', asn=216139)  # Find the AS node with ASN 216139
          .with_organizations()     # Include the organization that manages this AS
          .execute())
print(query1)

## 1. Basic Query - Find AS by Number

In [ ]:
# Find a specific AS (Google's AS15169) and retrieve its ASN and name
# Plain English: "Look for Google's Autonomous System (AS15169) and
# show me its AS number and name/description"
result = (iyp.builder()
          .find('AS', alias='google', asn=15169)  # Find AS15169, call it 'google'
          .return_fields(['google.asn', 'google.name'])  # Only return these fields
          .execute())

print(f"AS15169: {result[0]['google.name'] if result else 'Not found'}")
print(result)

## 2. Relationship Traversal - Find Upstream Providers

In [ ]:
# Find upstream providers (transit providers) for AS15169 up to 2 hops away
# Plain English: "Show me which networks provide internet connectivity to AS15169.
# Look up to 2 levels deep in the provider chain (provider's providers)"
# This helps identify who AS15169 depends on for internet connectivity
providers = iyp.find_upstream_providers(asn=15169, max_hops=2)

print("Upstream providers for AS15169:")
for p in providers[:3]:  # Show first 3
    print(f"  - AS{p.get('upstream.asn', 'N/A')}: {p.get('upstream.name', 'Unknown')}")

## 3. Complex Conditions - Boolean Logic

In [ ]:
# Find US-based ASes that depend on major transit providers
# Plain English: "Find all Autonomous Systems located in the United States
# that buy internet transit from one of the three major providers (Cogent, Level3, or Telia).
# This shows which US networks are customers of these major backbone providers."

major_transit = [174, 3356, 1299]  # Cogent (174), Level3 (3356), Telia (1299)

results = (iyp.builder()
           .find('AS', alias='customer')  # Find AS nodes, call them 'customer'
           .with_relationship('DEPENDS_ON', to='AS', alias='provider')  # Find their providers
           .with_relationship('COUNTRY', to='Country', alias='country')  # Find their country
           .where(
               And(  # Both conditions must be true:
                   Q('provider.asn').in_(major_transit),  # Provider is one of the major ones
                   Q('country.country_code') == 'US'      # Customer is in the US
               )
           )
           .limit(3)  # Only show first 3 results
           .return_fields(['customer.asn', 'customer.name', 'provider.asn'])
           .execute())

print(f"Found {len(results)} US customers of major transit providers")
for r in results:
    print(f"  AS{r['customer.asn']} -> AS{r['provider.asn']}")

## 4. Graph Traversal - Multi-hop Relationships

In [ ]:
# Find 2-hop upstream providers from AS216139
# Plain English: "Starting from AS216139, find their upstream providers (1 hop),
# then find those providers' upstream providers (2 hops). This shows the
# provider hierarchy two levels up - useful for understanding network dependencies
# and potential single points of failure in connectivity."

results = (iyp.builder()
           .find('AS', alias='start', asn=216139)  # Start from AS216139
           .upstream(hops=2, alias='upstream2')     # Go 2 hops upstream in provider chain
           .return_fields(['upstream2.asn', 'upstream2.name'])
           .limit(5)  # Show first 5 results
           .execute())

print("2-hop upstream from AS216139:")
for r in results:
    print(f"  AS{r['upstream2.asn']}: {r.get('upstream2.name', 'Unknown')}")

## 5. Export to DataFrame

In [ ]:
# Get UK-based ASes and export to pandas DataFrame
# Plain English: "Find all Autonomous Systems that are located in Great Britain (GB),
# retrieve their AS number and name, and return the results as a pandas DataFrame
# for easy data analysis and manipulation."

df = (iyp.builder()
      .find('AS', alias='uk_as')  # Find AS nodes, call them 'uk_as'
      .with_relationship('COUNTRY', to='Country', alias='country')  # Include country info
      .where(Q('country.country_code') == 'GB')  # Filter for Great Britain only
      .limit(10)  # Get first 10 results
      .return_fields(['uk_as.asn', 'uk_as.name', 'country.country_code'])
      .execute_df())  # Return as DataFrame instead of dictionary

print(f"UK ASes (showing {len(df)} rows):")
print(df[['uk_as.asn', 'uk_as.name']].head())

## 6. Debug - View Generated Cypher

In [ ]:
# Debug: View the generated Cypher query before execution
# Plain English: "Find AS15169 (Google) and all the ASes it peers with
# (direct connections for exchanging traffic). Show the raw Cypher query
# that would be sent to the Neo4j database - useful for debugging and
# understanding what's happening under the hood."

query = (iyp.builder()
         .find('AS', alias='test')  # Find AS nodes
         .with_relationship('PEERS_WITH', to='AS', alias='peer')  # Find peering relationships
         .where(Q('test.asn') == 15169)  # Filter for AS15169 (Google)
         .limit(2)  # Only show 2 peers
         .return_fields(['test.asn', 'peer.asn', 'peer.name']))

print("Generated Cypher:")
cypher, params = query.to_cypher()  # Convert to raw Cypher query
print(cypher)
print("\nParameters:")
print(params)

In [13]:
# Query 3: Find AS216139 and its managing organization, then show the generated Cypher
# Plain English: "Find AS216139 and the organization that manages it.
# Then display the raw Cypher query and parameters that would be sent to Neo4j.
# This demonstrates how the API translates our Python method calls into
# Neo4j's native Cypher query language."

query_3 = (iyp.builder()
            .find('AS', asn=216139)      # Find AS with ASN 216139
            .with_organizations()         # Include managing organization
            .upstream(hops=1, alias='up')
            )# Go 1 hop upstream in provider chain

print(query_3)  # Shows the query builder object

# Show the underlying Cypher query and parameters
cypher, param = query_3.to_cypher()
print(cypher, param)

MATCH (as:AS {asn: $param_0})
MATCH (as)-[:MANAGED_BY]->(organization:Organization)
MATCH (as)-[:DEPENDS_ON]->(up:AS)
RETURN up, organization, as {'param_0': 216139}


## Summary of Essential Features

✅ **Basic Queries** - Find nodes by properties  
✅ **Relationships** - Traverse graph connections  
✅ **Boolean Logic** - Complex AND/OR/NOT conditions  
✅ **Multi-hop Traversal** - Follow relationships multiple steps  
✅ **DataFrame Export** - Integration with pandas  
✅ **Debug Support** - View generated Cypher queries  